# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [1]:
def build_kernel(data_set, kernel_type='linear', d_poly_exponential=1):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = np.power(np.dot(data_set, data_set.T), d_poly_exponential)
        #the formula given in the task is wrong, should be K_ij = (x^T x)_{ij}^d, otherwise it is not a matrix. 
        #In particular
    return kernel

The formula given in the task is wrong, should be $K_{ij} = ((x^T x)_{ij})^d$, otherwise it is not a matrix, and at first sight it is not obvious that d is the power applied elementwise and not a multiplication of matrices. 

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [84]:
def choose_set_for_label(data_set, label, labels):
    # your code here
    #ONE VERSUS ALL METHOD
    labels_local = np.copy(labels)
    
    labels_local[labels!=label] = -1
    labels_local[labels==label] = 1

    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
        data_set, labels_local, test_size=0.2, random_state=10) 
    #every invokation the test and train sets are based on the same datapoints

    return train_data_set, test_data_set, train_labels, test_labels

In [85]:
def get_labels_count(labels):
    labels_count = len(np.unique(labels))
    #in fact I don't know what this function should do
    return labels_count

Use the code that we have implemented earlier:

In [86]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)
    objects_count = len(train_labels)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [87]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt
from sklearn.metrics import accuracy_score

iris = load_iris()
data_set = iris.data
labels = iris.target


train_data_set_all, test_data_set_all, train_labels_all, test_labels_all = train_test_split(
        data_set, labels, test_size=0.2, random_state=10) 

(train_data_set, test_data_set, train_labels, test_labels) = choose_set_for_label(data_set, labels[0], labels)
predicted_one_vs_all = np.empty((0, len(test_labels)))

for label in np.unique(labels):
    # modify this part 
    (train_data_set, test_data_set, train_labels, test_labels) = choose_set_for_label(data_set, label, labels)
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
    pred_rbf = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    pred_rbf = pred_rbf.astype(int)
    #print(accuracy_score(pred_rbf, test_labels))
    predicted_one_vs_all = np.vstack((predicted_one_vs_all, np.ndarray.flatten(pred_rbf)))

    

#ONE VS ALL VOTING
#if we have only three classes, then one vs all voting is simply selecting the class 
#that was classified as 1 and not -1. If there is for example three 1's for all three labels,
#the choice should be random. Here we select the third "3" label by deafult in such case. 
#If we have three -1's, the first class "0" wins by default. 
#The second label is selected if we have for example (1,1,-1) predictions.

predicted = np.zeros((len(predicted_one_vs_all[0])))
for label_idx, predicted_label_vs_all in enumerate(predicted_one_vs_all):
    label = np.unique(labels)[label_idx]
    predicted[predicted_label_vs_all > -1] = label
    #print((label,predicted_label_vs_all))

    
print("Prediction on test data: ", predicted)
print("True test data: ", test_labels_all)
print("Accuracy: ", accuracy_score(predicted, test_labels_all))

     pcost       dcost       gap    pres   dres
 0:  1.1709e+02 -1.8305e+03  4e+03  2e-01  2e-15
 1:  7.8677e+01 -1.7328e+02  3e+02  6e-03  3e-15
 2:  1.0363e+01 -2.2583e+01  3e+01  1e-15  3e-15
 3: -4.6904e-01 -5.0369e+00  5e+00  5e-16  2e-15
 4: -1.4254e+00 -2.5039e+00  1e+00  2e-16  5e-16
 5: -1.6760e+00 -2.1752e+00  5e-01  2e-16  3e-16
 6: -1.8059e+00 -2.0041e+00  2e-01  2e-16  3e-16
 7: -1.8649e+00 -1.9581e+00  9e-02  4e-16  3e-16
 8: -1.8911e+00 -1.8991e+00  8e-03  5e-16  3e-16
 9: -1.8944e+00 -1.8945e+00  1e-04  2e-16  3e-16
10: -1.8945e+00 -1.8945e+00  2e-06  7e-16  3e-16
11: -1.8945e+00 -1.8945e+00  4e-08  8e-16  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.1393e+02 -2.4094e+03  5e+03  2e-01  2e-15
 1:  9.0906e+01 -2.1667e+02  4e+02  6e-03  3e-15
 2:  1.2607e+01 -2.5542e+01  4e+01  3e-05  4e-15
 3: -1.4285e-01 -5.4680e+00  5e+00  2e-16  2e-15
 4: -1.3471e+00 -2.2723e+00  9e-01  4e-16  6e-16
 5: -1.5481e+00 -1.9653e+00  4e-01  4e-16  3e-1


The resulting accuracy 56.6% is better than just guessing (33.3%).
